In [1]:
import cv2 as cv
import pandas as pd
import os
import pickle

# Process Data

In [2]:
dirname = 'data/raw/'

In [3]:
def getListOfFiles(dirName):
    # adapted from https://thispointer.com/python-how-to-get-list-of-files-in-directory-and-sub-directories/
    # create a list of file and sub directories names in the given directory  
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [4]:
allFiles = getListOfFiles(dirname)

In [5]:
new_dim = (80,80)

for filename in allFiles:
    # Reading BGR
    img = cv.imread(filename)
    # Resizing
    img = img[:,64:448,:]
    img = cv.resize(img, new_dim, interpolation = cv.INTER_AREA)
    # Writing BGR
    cv.imwrite(filename.replace("raw","processed"), img)

# Dataframe construction from folder structure

In [6]:
class_one_hot = {
    "cardboard":[1,0,0,0,0],
    "glass":[0,1,0,0,0],    
    "metal":[0,0,1,0,0],    
    "paper":[0,0,0,1,0],
    "plastic": [0,0,0,0,1]
}

In [7]:
columns = ["filename","class_label","class_one_hot"]
df = pd.DataFrame(columns=columns)

class_path_folder = "data/processed/"
directory = os.listdir(class_path_folder)
i=0
# Iterate over all the entries
for class_label in directory:
    # Create full path
    fullPath_class = os.path.join(class_path_folder, class_label)
    if os.path.isdir(fullPath_class):
        for img in os.listdir(fullPath_class):
            filename = os.path.join(fullPath_class, img)
            if os.path.isfile(filename):
                df.loc[i] = [filename, class_label, class_one_hot[class_label]]
                i+=1 

In [8]:
df.head()

,filename,class_label,class_one_hot
0,data/processed/plastic/plastic80.jpg,plastic,"[0, 0, 0, 0, 1]"
1,data/processed/plastic/plastic381.jpg,plastic,"[0, 0, 0, 0, 1]"
2,data/processed/plastic/plastic169.jpg,plastic,"[0, 0, 0, 0, 1]"
3,data/processed/plastic/plastic354.jpg,plastic,"[0, 0, 0, 0, 1]"
4,data/processed/plastic/plastic185.jpg,plastic,"[0, 0, 0, 0, 1]"


In [9]:
save_dataframe = 1
if save_dataframe:
    df.to_pickle("/mmdetection/notebooks/Trashnet/repo/data/df.pickle")